<a href="https://colab.research.google.com/github/Garrafao/WUGs/blob/main/scripts/misc/wic2wug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://pilehvar.github.io/wic/package/WiC_dataset.zip

--2023-05-12 08:08:36--  https://pilehvar.github.io/wic/package/WiC_dataset.zip
Resolving pilehvar.github.io (pilehvar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to pilehvar.github.io (pilehvar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275984 (270K) [application/zip]
Saving to: ‘WiC_dataset.zip’

WiC_dataset.zip     100%[===================>] 269.52K  --.-KB/s    in 0.02s   

2023-05-12 08:08:36 (12.9 MB/s) - ‘WiC_dataset.zip’ saved [275984/275984]



In [ ]:
import pandas as pd
import numpy as np
import os
import io
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
directory = os.makedirs( '/content/WIC_data', exist_ok=True )

In [ ]:
dev = os.makedirs( '/content/WIC/dev', exist_ok=True )
train = os.makedirs( '/content/WIC/train', exist_ok=True )
test = os.makedirs( '/content/WIC/test', exist_ok=True )


In [ ]:
#path = "/content/dev/dev.data.txt"

In [ ]:
#directory = path.split('/')[2]

In [ ]:
!pip install spacy  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from zipfile import ZipFile
with ZipFile('WiC_dataset.zip', 'r') as WiC:
   WiC.extractall()

In [ ]:
df_dev = pd.read_csv("/content/dev/dev.data.txt", sep = '\t', names=["lemma", "pos", "index", "sent1", "sent2"], encoding = 'UTF-8')
df_train = pd.read_csv("/content/train/train.data.txt", sep = '\t', names=["lemma", "pos", "index", "sent1", "sent2"], encoding = 'UTF-8')
df_test = pd.read_csv("/content/test/test.data.txt", sep = '\t', names=["lemma", "pos", "index", "sent1", "sent2"], encoding = 'UTF-8')


In [ ]:
dev_label= pd.read_csv("/content/dev/dev.gold.txt", sep = '\t', names = ['judgment'], encoding = 'UTF-8')
train_label= pd.read_csv("/content/train/train.gold.txt", sep = '\t', names = ['judgment'], encoding = 'UTF-8')
test_label= pd.read_csv("/content/test/test.gold.txt", sep = '\t', names = ['judgment'], encoding = 'UTF-8')

In [ ]:
x= df_dev['index'].tolist()
y = df_train['index'].tolist()
z = df_test['index'].tolist()

In [ ]:
df_dash = df_dev[['lemma', 'sent1', 'sent2']]
df_dash1 = df_train[['lemma', 'sent1', 'sent2']]
df_dash2 = df_test[['lemma', 'sent1', 'sent2']]

In [ ]:
df1 = df_dash[['lemma', 'sent1']]
df1.columns =  ['word', 'sent']
df1["index"] = df1.groupby("word").cumcount()*2

dfone = df_dash1[['lemma', 'sent1']]
dfone.columns =  ['word', 'sent']
dfone["index"] = dfone.groupby("word").cumcount()*2

dfein = df_dash2[['lemma', 'sent1']]
dfein.columns =  ['word', 'sent']
dfein["index"] = dfein.groupby("word").cumcount()*2

In [ ]:
df2 =  df_dash[['lemma', 'sent2']]
df2.columns =  ['word', 'sent']
df2["index"] = df2.groupby("word").cumcount()*2+1

dftwo =  df_dash1[['lemma', 'sent2']]
dftwo.columns =  ['word', 'sent']
dftwo["index"] = dftwo.groupby("word").cumcount()*2+1

dfdu =  df_dash2[['lemma', 'sent2']]
dfdu.columns =  ['word', 'sent']
dfdu["index"] = dfdu.groupby("word").cumcount()*2+1

In [ ]:
df_final_dev = pd.concat([df1, df2])
df_final_dev = df_final_dev.sort_values(by = ['word', 'index'], ascending = [True, True]) 

df_final_train = pd.concat([dfone, dftwo])
df_final_train = df_final_train.sort_values(by = ['word', 'index'], ascending = [True, True])

df_final_test = pd.concat([dfein, dfdu])
df_final_test = df_final_test.sort_values(by = ['word', 'index'], ascending = [True, True])

In [ ]:
df_final_test

,word,sent,index
59,abort,I wasted a year of my life working on an abort .,0
59,abort,He sent a short message requesting an abort du...,1
1095,absorption,"The absorption of light , heat , electricity ,...",0
1095,absorption,The absorption of photons by atoms or molecules .,1
618,accept,To accept the report of a committee .,0
...,...,...,...
1224,zero,Write 0.0 to indicate a floating point number ...,5
1023,zinger,She tried to think of some killer of an argume...,0
1023,zinger,He always greeted me with a new zinger .,1
169,zone,The white zone is for loading and unloading on...,0


In [ ]:
df_final_dev["identifier"] = df_final_dev["word"]+"-"+df_final_dev["index"].astype(str)
df_final_train["identifier"] = df_final_train["word"]+"-"+df_final_train["index"].astype(str)
df_final_test["identifier"] = df_final_test["word"]+"-"+df_final_test["index"].astype(str)

In [ ]:
df_odd = df_final_dev[1::2]
df_odd.columns = ["word2", "sent2", "index2", "identifier2"]
df_even = df_final_dev[::2]
df_even.columns = ["word1", "sent1", "index1", "identifier1"]
df_final_dev = pd.concat([df_even, df_odd], axis=1)

df_odd1 = df_final_train[1::2]
df_odd1.columns = ["word2", "sent2", "index2", "identifier2"]
df_even1 = df_final_train[::2]
df_even1.columns = ["word1", "sent1", "index1", "identifier1"]
df_final_train = pd.concat([df_even1, df_odd1], axis=1)

df_odd2 = df_final_test[1::2]
df_odd2.columns = ["word2", "sent2", "index2", "identifier2"]
df_even2 = df_final_test[::2]
df_even2.columns = ["word1", "sent1", "index1", "identifier1"]
df_final_test = pd.concat([df_even2, df_odd2], axis=1)

In [ ]:
df_final_dev = df_final_dev[["word2", "sent2", "identifier2", "sent1", "identifier1", 'index1', 'index2']]
df_final_dev = df_final_dev.rename(columns={"word2":"lemma"})

df_final_train = df_final_train[["word2", "sent2", "identifier2", "sent1", "identifier1", 'index1', 'index2']]
df_final_train = df_final_train.rename(columns={"word2":"lemma"})

df_final_test = df_final_test[["word2", "sent2", "identifier2", "sent1", "identifier1", 'index1', 'index2']]
df_final_test = df_final_test.rename(columns={"word2":"lemma"})

In [ ]:
df_final_dev["judgment"] = dev_label["judgment"].values.tolist()
df_final_train["judgment"] = train_label["judgment"].values.tolist()
df_final_test["judgment"] = test_label["judgment"].values.tolist()

In [ ]:
df_final_dev = df_final_dev[['lemma', 'identifier2', 'identifier1', 'judgment']]
df_final_train = df_final_train[['lemma', 'identifier2', 'identifier1', 'judgment']]
df_final_test = df_final_test[['lemma', 'identifier2', 'identifier1', 'judgment']]

In [ ]:
df_final_dev["annotator"] = "gold"
df_final_dev['comment'] = " "
df_final_train["annotator"] = "gold"
df_final_train['comment'] = " "
df_final_test["annotator"] = "gold"
df_final_test['comment'] = " "

In [ ]:
df_final_dev = df_final_dev[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma"]]
df_final_train = df_final_train[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma"]]
df_final_test = df_final_test[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma"]]

In [ ]:
df_final_dev = df_final_dev.reset_index(drop =True)
df_final_train = df_final_train.reset_index(drop =True)
df_final_test = df_final_test.reset_index(drop =True)

In [ ]:
for i in list(df_final_dev["lemma"].value_counts().index):
        tmp_df = df_final_dev[df_final_dev["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/dev'+ "/"+i):
            os.mkdir('/content/WIC/dev'+ "/"+i)
        np.savetxt('/content/WIC/dev' +"/"+i+"/judgments.csv", numpy_df,fmt='%s', delimiter='\t')

In [ ]:
for i in list(df_final_train["lemma"].value_counts().index):
        tmp_df = df_final_train[df_final_train["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/train'+ "/"+i):
            os.mkdir('/content/WIC/train'+ "/"+i)
        np.savetxt('/content/WIC/train' +"/"+i+"/judgments.csv", numpy_df,fmt='%s', delimiter='\t')

In [ ]:
for i in list(df_final_test["lemma"].value_counts().index):
        tmp_df = df_final_test[df_final_test["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/test'+ "/"+i):
            os.mkdir('/content/WIC/test'+ "/"+i)
        np.savetxt('/content/WIC/test' +"/"+i+"/judgments.csv", numpy_df,fmt='%s', delimiter='\t')

In [ ]:
ind1 = []
ind2 = []
for i in x:
    ind1.append(i.split('-')[0])
    ind2.append(i.split('-')[1])

ind_1 = []
ind_2 = []
for i in y:
    ind_1.append(i.split('-')[0])
    ind_2.append(i.split('-')[1])

indone = []
indtwo = []
for i in z:
    indone.append(i.split('-')[0])
    indtwo.append(i.split('-')[1])

In [ ]:
df_dev.loc[:,'ind1'] = ind1
df_dev.loc[:,'ind2'] = ind2

df_train.loc[:,'ind_1'] = ind_1
df_train.loc[:,'ind_2'] = ind_2

df_test.loc[:,'indone'] = indone
df_test.loc[:,'indtwo'] = indtwo

In [ ]:
df1 = df_dev[['lemma', 'pos', 'sent1', 'ind1']]
df1.columns =  ['lemma','pos', 'context_tokenized','ind']
df10 = df_train[['lemma', 'pos', 'sent1', 'ind_1']]
df10.columns =  ['lemma','pos', 'context_tokenized','ind']
df100 = df_test[['lemma', 'pos', 'sent1', 'indone']]
df100.columns =  ['lemma','pos', 'context_tokenized','ind']

In [ ]:
df2 = df_dev[['lemma', 'pos', 'sent2', 'ind2']]
df2.columns =  ['lemma','pos', 'context_tokenized','ind']
df20 = df_train[['lemma', 'pos', 'sent2', 'ind_2']]
df20.columns =  ['lemma','pos', 'context_tokenized','ind']
df200 = df_test[['lemma', 'pos', 'sent2', 'indtwo']]
df200.columns =  ['lemma','pos', 'context_tokenized','ind']

In [ ]:
df_final_dev_uses = pd.concat([df1, df2])
df_final_dev_uses = df_final_dev_uses.sort_values(by = ['lemma'], ascending = True)
df_final_train_uses = pd.concat([df10, df20])
df_final_train_uses= df_final_train_uses.sort_values(by = ['lemma'], ascending = True)
df_final_test_uses = pd.concat([df100, df200])
df_final_test_uses = df_final_test_uses.sort_values(by = ['lemma'], ascending = True)


In [ ]:
df_final_dev_uses = df_final_dev_uses.reset_index(drop =True)
df_final_train_uses = df_final_train_uses.reset_index(drop =True)
df_final_test_uses = df_final_test_uses.reset_index(drop =True)

In [ ]:
def get_indice(sent, indice):
  tag = ''
  indice = int(indice)
  tok = sent.split(" ")
  #print(tok)
  for i in range(len(tok)):
    if i == indice:
      tag = tok[i]
  return str(sent.find(tag))+":"+str(sent.find(tag)+len(tag))

In [ ]:
df_final_dev_uses["indexes_target_token"] = df_final_dev_uses.apply(lambda x: get_indice(x.context_tokenized, x.ind), axis=1)
df_final_train_uses["indexes_target_token"] = df_final_train_uses.apply(lambda x: get_indice(x.context_tokenized, x.ind), axis=1)
df_final_test_uses["indexes_target_token"] = df_final_test_uses.apply(lambda x: get_indice(x.context_tokenized, x.ind), axis=1)

In [ ]:
def get_indices_of_sent(sentence):
    return "0:"+str(len(sentence))

In [ ]:
df_final_dev_uses["indexes_target_sentence"] = df_final_dev_uses["context_tokenized"].apply(get_indices_of_sent)
df_final_train_uses["indexes_target_sentence"] = df_final_train_uses["context_tokenized"].apply(get_indices_of_sent)
df_final_test_uses["indexes_target_sentence"] = df_final_test_uses["context_tokenized"].apply(get_indices_of_sent)

In [ ]:
def get_len_tok(sentence):
  return "0:"+str(len(sentence.split(" ")))

In [ ]:
df_final_dev_uses['indexes_target_sentence_tokenized'] = df_final_dev_uses["context_tokenized"].apply(get_len_tok)
df_final_train_uses['indexes_target_sentence_tokenized'] = df_final_train_uses["context_tokenized"].apply(get_len_tok)
df_final_test_uses['indexes_target_sentence_tokenized'] = df_final_test_uses["context_tokenized"].apply(get_len_tok)

In [ ]:
lem = ''
def lemmatizr(sent):
  doc = nlp(sent)
  for i in doc:
    lem += i.lemma_
    return lem

In [ ]:
df_final_dev_uses['context_lemmatized'] = df_final_dev_uses["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [ ]:
df_final_train_uses['context_lemmatized'] = df_final_train_uses["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
df_final_test_uses['context_lemmatized'] = df_final_test_uses["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [ ]:
df_final_dev_uses["index"] = df_final_dev_uses.groupby("lemma").cumcount()
df_final_test_uses["index"] = df_final_test_uses.groupby("lemma").cumcount()
df_final_train_uses["index"] = df_final_train_uses.groupby("lemma").cumcount()

In [ ]:
df_final_dev_uses['context_pos'] = df_final_dev_uses["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))
df_final_train_uses['context_pos'] = df_final_train_uses["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))
df_final_test_uses['context_pos'] = df_final_test_uses["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))

In [ ]:
df_final_dev_uses['date'] = " "
df_final_dev_uses['grouping'] = 1
df_final_dev_uses['description'] = " "
df_final_dev_uses['context'] = df_final_dev_uses['context_tokenized']
#df_final_dev_uses['identifier'] = ""
df_final_dev_uses['indexes_target_token_tokenized'] = df_final_dev_uses['ind']

df_final_train_uses['date'] = " "
df_final_train_uses['grouping'] = 1
df_final_train_uses['description'] = " "
df_final_train_uses['context'] = df_final_train_uses['context_tokenized']
#df_final_train_uses['identifier'] = 
df_final_train_uses['indexes_target_token_tokenized'] = df_final_train_uses['ind']

df_final_test_uses['date'] = " "
df_final_test_uses['grouping'] = 1
df_final_test_uses['description'] = " "
df_final_test_uses['context'] = df_final_test_uses['context_tokenized']
#df_final_test_uses['identifier'] = 
df_final_test_uses['indexes_target_token_tokenized'] = df_final_test_uses['ind']

df_final_dev_uses["identifier"] = df_final_dev_uses["lemma"]+"-"+df_final_dev_uses["index"].astype(str)
df_final_train_uses["identifier"] = df_final_train_uses["lemma"]+"-"+df_final_train_uses["index"].astype(str)
df_final_test_uses["identifier"] = df_final_test_uses["lemma"]+"-"+df_final_test_uses["index"].astype(str)

In [ ]:
final_df_dev = df_final_dev_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos']]
final_df_train = df_final_train_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos']]
final_df_test = df_final_test_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos']]

In [62]:
!git clone https://github.com/Garrafao/WUGs.git

Cloning into 'WUGs'...
remote: Enumerating objects: 1364, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 1364 (delta 158), reused 269 (delta 146), pack-reused 1074
Receiving objects: 100% (1364/1364), 1.76 MiB | 11.42 MiB/s, done.
Resolving deltas: 100% (836/836), done.


In [64]:
os.makedirs('/content/WUGs/scripts/misc/data/dwug_en/data/*/', exist_ok=True )

In [65]:
import csv
final_df_dev.to_csv('/content/WUGs/scripts/misc/data/dwug_en/data/*/uses_dev.csv', sep='\t', encoding='utf-8', quoting = csv.QUOTE_NONE, quotechar='')
final_df_train.to_csv('/content/WUGs/scripts/misc/data/dwug_en/data/*/uses_train.csv', sep='\t', encoding='utf-8', quoting = csv.QUOTE_NONE, quotechar='')
final_df_test.to_csv('/content/WUGs/scripts/misc/data/dwug_en/data/*/uses_test.csv', sep='\t', encoding='utf-8', quoting = csv.QUOTE_NONE, quotechar='')


In [66]:
%run /content/WUGs/scripts/misc/use2normalize.py /content/WUGs/scripts/misc/data/dwug_en/data/*/uses_dev.csv dwug_en output_dev

target:  abatement
target:  abatement
target:  abridged
target:  abridge
target:  absorbed
target:  Absorb
target:  abuse
target:  abuse
target:  Accession
target:  accessions
target:  accumulate
target:  accumulating
target:  acquaintance
target:  acquaintances
target:  acquisition
target:  acquisition
target:  act
target:  acting
target:  activity
target:  activity
target:  activity
target:  activity
target:  addition
target:  addition
target:  adhocracy
target:  adhocracy
target:  admitted
target:  Admit
target:  Adoxography
target:  adoxography
target:  advantage
target:  advantage
target:  affair
target:  affair
target:  affect
target:  affects
target:  affiliation
target:  affiliation
target:  afflicted
target:  Afflict
target:  afford
target:  afford
target:  agency
target:  agency
target:  Agency
target:  Agency
target:  aim
target:  aimed
target:  aliyah
target:  aliyah
target:  allowance
target:  allowanced
target:  allowance
target:  allowance
target:  amuse
target:  amused


In [67]:
%run /content/WUGs/scripts/misc/use2normalize.py /content/WUGs/scripts/misc/data/dwug_en/data/*/uses_train.csv dwug_en output_train

Streaming output truncated to the last 5000 lines.
target:  Make
target:  make
target:  Make
target:  Make
target:  made
target:  make
target:  Make
target:  make
target:  Make
target:  make
target:  make
target:  made
target:  Make
target:  make
target:  make
target:  make
target:  make
target:  make
target:  make
target:  make
target:  made
target:  made
target:  makes
target:  make
target:  Make
target:  make
target:  make
target:  make
target:  make
target:  Make
target:  make
target:  made
target:  Make
target:  Make
target:  make
target:  makes
target:  Make
target:  made
target:  makes
target:  makes
target:  made
target:  Make
target:  Make
target:  made
target:  Make
target:  Make
target:  make
target:  make
target:  Make
target:  made
target:  made
target:  Make
target:  Make
target:  make
target:  Make
target:  make
target:  make
target:  Make
target:  make
target:  Make
target:  make
target:  Make
target:  make
target:  Make
target:  made
target:  make
target:  makes
target

In [68]:
%run /content/WUGs/scripts/misc/use2normalize.py /content/WUGs/scripts/misc/data/dwug_en/data/*/uses_test.csv dwug_en output_test

target:  abort
target:  abort
target:  absorption
target:  absorption
target:  accepted
target:  accept
target:  access
target:  access
target:  account
target:  account
target:  account
target:  account
target:  acknowledged
target:  acknowledged
target:  acknowledge
target:  acknowledge
target:  acquire
target:  acquired
target:  act
target:  acting
target:  action
target:  action
target:  active
target:  active
target:  adds
target:  add
target:  addition
target:  addition
target:  adhered
target:  adhere
target:  adhesion
target:  adhesion
target:  adjoin
target:  adjoin
target:  Administer
target:  Administer
target:  Administer
target:  administers
target:  affinity
target:  affinity
target:  Afforest
target:  afforest
target:  afterthought
target:  afterthought
target:  aged
target:  Age
target:  age
target:  age
target:  agency
target:  Agency
target:  aggravate
target:  aggravate
target:  agony
target:  agony
target:  agreed
target:  agree
target:  agreed
target:  agree
target

In [69]:
final_u_dev = pd.read_csv('/content/output_dev', delimiter = '\t',quoting =3)
final_u_train = pd.read_csv('/content/output_train', delimiter = '\t',quoting =3)
final_u_test = pd.read_csv('/content/output_test', delimiter = '\t',quoting =3)

In [75]:
final_u_dev['context_tokenized'] = final_df_dev['context_tokenized']
final_u_dev['pos'] = final_df_dev['pos']
#final_u_dev['dataset'] = final_df_dev['dataset']
final_u_dev['grouping'] = final_df_dev['grouping']
final_u_dev['lemma'] = final_df_dev['lemma']
final_u_dev['indexes_target_token_tokenized'] = final_df_dev['indexes_target_token_tokenized']
final_u_dev['indexes_target_sentence_tokenized'] = final_df_dev['indexes_target_sentence_tokenized']
final_u_dev['date'] = " "
final_u_dev['description'] = " "
final_u_dev['context_lemmatized'] = final_df_dev['context_lemmatized']
final_u_dev['context_pos'] = final_df_dev['context_pos']


final_u_train['context_tokenized'] = final_df_train['context_tokenized']
final_u_train['pos'] = final_df_train['pos']
#final_u_dev['dataset'] = final_df_dev['dataset']
final_u_train['grouping'] = final_df_train['grouping']
final_u_train['lemma'] = final_df_train['lemma']
final_u_train['indexes_target_token_tokenized'] = final_df_train['indexes_target_token_tokenized']
final_u_train['indexes_target_sentence_tokenized'] = final_df_train['indexes_target_sentence_tokenized']
final_u_train['date'] = " "
final_u_train['description'] = " "
final_u_train['context_lemmatized'] = final_df_train['context_lemmatized']
final_u_train['context_pos'] = final_df_train['context_pos']

final_u_test['context_tokenized'] = final_df_test['context_tokenized']
final_u_test['pos'] = final_df_test['pos']
#final_u_dev['dataset'] = final_df_dev['dataset']
final_u_test['grouping'] = final_df_test['grouping']
final_u_test['lemma'] = final_df_test['lemma']
final_u_test['indexes_target_token_tokenized'] = final_df_test['indexes_target_token_tokenized']
final_u_test['indexes_target_sentence_tokenized'] = final_df_test['indexes_target_sentence_tokenized']
final_u_test['date'] = " "
final_u_test['description'] = " "
final_u_test['context_lemmatized'] = final_df_test['context_lemmatized']
final_u_test['context_pos'] = final_df_test['context_pos']



In [76]:
final_u_train = final_u_train[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos']]
final_u_test = final_u_test[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos']]
final_u_dev = final_u_dev[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos']]

In [79]:
for i in list(final_u_dev["lemma"].value_counts().index):
        tmp_df = final_u_dev[final_u_dev["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/dev'+"/"+i):
            os.mkdir('/content/WIC/dev'+"/"+i)
        np.savetxt('/content/WIC/dev'+"/"+i+"/uses.csv", numpy_df,fmt='%s', delimiter='\t')

In [80]:
for i in list(final_u_train["lemma"].value_counts().index):
        tmp_df = final_u_train[final_u_train["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/train'+"/"+i):
            os.mkdir('/content/WIC/train'+"/"+i)
        np.savetxt('/content/WIC/train'+"/"+i+"/uses.csv", numpy_df,fmt='%s', delimiter='\t')

In [81]:
for i in list(final_u_test["lemma"].value_counts().index):
        tmp_df = final_u_test[final_u_test["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/test'+"/"+i):
            os.mkdir('/content/WIC/test'+"/"+i)
        np.savetxt('/content/WIC/test'+"/"+i+"/uses.csv", numpy_df,fmt='%s', delimiter='\t')

In [ ]:
#c = final_df.context.tolist()

In [ ]:
#y = []
#for m in c:
 # y.append(m.split(" "))

In [ ]:
#lengthh = []
#for i in y:
 #   lengthh.append(len(i))

In [ ]:
#final_df.loc[:,'lengthh'] = lengthh

In [ ]:
#d = final_df.context_pos.tolist()

In [ ]:
#z = []
#for m in d:
 # z.append(m.split(" "))

In [ ]:
#lengthh2 = []
#for i in z:
 #   lengthh2.append(len(i))

In [ ]:
#final_df.loc[:,'lengthh2'] = lengthh2